## Objective
Parses a text file table that has reinsurance and risk adjustment data for a benefit year into a CSV.

### Preprocessing
step 1: copy into raw text file

step 2: delete header 

HIOS ID
HIOS INPUTTED
INSURANCE COMPANY
NAME STATE
REINSURANCE
PAYMENT
AMOUNT (OR
NOT ELIGIBLE)
HHS RISK
ADJUSTMENT
TRANSFER
AMOUNT
(INDIVIDUAL
MARKET,
INCLUDING
CATASTROPHIC)
HHS RISK
ADJUSTMENT
TRANSFERS
AMOUNT (SMALL
GROUP MARKET)

step 3: delete page numbers through the regexp ^RI RA.\*

step 4: remove blank lines through the regexp ^\n\n

step 5: find and replace "not eligible" with "N/E"

step 6: find and replace "\$ -" with "\$-"

step 7: find and replace "\$ " with "\$" (also works for step 6)

step 8: find and replace "N/A - MA Issuer" with "N/A_MA_Issuer" (dataframe has trouble parsing "-")

step 9: find and replace "N/A - Default Charge" with "N/A_DefaultCharge"

step 10: delete footnotes on the first page

In [1]:
import pandas as pd
# import re

In [2]:
#with open('2014-Benefit-Year-Edit.txt', 'r') as myfile:
with open('Input/2018-PDF.txt', 'r') as myfile:
    s = myfile.read()

In [3]:
split = str.split(s)
length = len(split)

In [4]:
storage = pd.DataFrame(columns=["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME", "STATE",
                                "HIGH-COST RISK POOL PAYMENT AMOUNT (INDIVIDUAL (Catastrophic and NonCatastrophic))",
                                "HIGH-COST RISK POOL PAYMENT AMOUNT (SMALL GROUP)",
                                "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL NONCATASTROPHIC RISK POOL)",
                                "HHS RISK ADJUSTMENT TRANSFER AMOUNT (CATASTROPHIC RISK POOL)",
                                "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP RISK POOL)"])

# prog = re.compile('^[A-Z]{2}$')

i = 0
index = 0
while i < length - 1:
    hios_id = split[i]
    i = i + 1
    temp = i
    # while (prog.match(split[i]) == None or split[i] in ['US', 'BC', 'OF']): # look for state in quotes
    while (split[i][:1] != '(' and split[i][:1] != '$'):
        i = i + 1
    company_name = str(' '.join(split[temp:i - 1]))
    state = split[i - 1]
    high_cost_i = split[i]
    i = i + 1
    high_cost_sg = split[i]
    i = i + 1
    individual_noncat = split[i]
    i = i + 1
    individual_cat = split[i]
    i = i + 1
    small_group = split[i] 
    i = i + 1
    storage.loc[index] = [hios_id, company_name, state, high_cost_i, high_cost_sg,
                          individual_noncat, individual_cat, small_group]
    index = index + 1
    
storage.head()

,HIOS ID,HIOS INPUTTED INSURANCE COMPANY NAME,STATE,HIGH-COST RISK POOL PAYMENT AMOUNT (INDIVIDUAL (Catastrophic and NonCatastrophic)),HIGH-COST RISK POOL PAYMENT AMOUNT (SMALL GROUP),HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL NONCATASTROPHIC RISK POOL),HHS RISK ADJUSTMENT TRANSFER AMOUNT (CATASTROPHIC RISK POOL),HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP RISK POOL)
0,11082,"Aetna, Inc.",AK,$-,$0.00,$-,$-,"$40,127.64"
1,38344,Premera Blue Cross,AK,"$4,941,673.00","$644,483.75",$0.00,$-,"$124,518.87"
2,73836,"Moda Health Plan, Inc.",AK,$-,$0.00,$-,$-,"($368,683.53)"
3,80049,"UnitedHealth Group, Inc.",AK,$-,$0.00,$-,$-,"$204,037.00"
4,46944,Blue Cross Blue Shield of Alabama,AL,"$2,758,305.21","$516,017.10","$2,970,457.93","$13,570.07","$2,587,809.53"


In [5]:
storage.tail()

,HIOS ID,HIOS INPUTTED INSURANCE COMPANY NAME,STATE,HIGH-COST RISK POOL PAYMENT AMOUNT (INDIVIDUAL (Catastrophic and NonCatastrophic)),HIGH-COST RISK POOL PAYMENT AMOUNT (SMALL GROUP),HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL NONCATASTROPHIC RISK POOL),HHS RISK ADJUSTMENT TRANSFER AMOUNT (CATASTROPHIC RISK POOL),HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP RISK POOL)
531,72982,The Health Plan of the Upper Ohio Valley,WV,$0.00,$0.00,"($30,861.62)",$-,"($261,919.75)"
532,77060,"UnitedHealth Group, Inc.",WV,$-,$0.00,$-,$-,"($1,908,349.20)"
533,95628,"UnitedHealth Group, Inc.",WV,$-,$0.00,$-,$-,"($247,813.25)"
534,11269,Blue Cross Blue Shield of Wyoming,WY,"$177,831.59","$759,088.07",$0.00,$-,"$468,478.40"
535,49714,"UnitedHealth Group, Inc.",WY,$-,"$725,347.34",$-,$-,"($468,478.46)"


In [6]:
# save it
storage.to_csv("Input/2018-parsed-PDF.csv")